# TakagiTaupin - Calculate the Takagi-Taupin curves

This file demonstrates the usage of TakagiTaupin that is the main class of the package. TakagiTaupin solves the TT-equations for deformed crystal using the information given to it in TTcrystal and TTscan. For detailed instruction on their use, please refer to the respective documentation and example Jupyter notebooks.

In [1]:
import sys
import os.path

import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(1, '..')
from pyTTE import TakagiTaupin, TTcrystal, TTscan, Quantity

SyntaxError: name 'output_log' is used prior to global declaration (pyTTE.py, line 511)

### First example: rocking curve of a perfect Si crystal in the Bragg case

Let us start with a simple example of calculating the rocking curve of the (111) reflection of a 1 mm thick Si perfect, single crystal. First we define the crystal using TTcrystal class:

In [ ]:
xtal = TTcrystal(crystal='Si', hkl=[1,1,1], thickness=Quantity(1,'mm'))

The given keywords `crystal`, `hkl`, and `thickness` are the required minimum amount of parameters to define a crystal. To avoid conversion errors, all the quantities with physical units are handled in _pyTTE_ using the Quantity class which supports most units commonly used in the context of X-ray diffraction.

Full information of a TTcrystal instance can be seen by passing it to the `print` function:

In [ ]:
print(xtal)

In addition to the crystal parameters, we need to define the scan using the TTscan class. Let's fix the photon energy of the incident beam to 5 keV, use $\sigma$-polarization and scan the rocking angle from -50 to 150 µrad in 150 steps. In _pyTTE_, all scans are performed relative to the values fulfilling the kinematical Bragg condition $\lambda = 2 d \sin \theta$.  

In [ ]:
scan = TTscan(constant = Quantity(5,'keV'), scan = Quantity(np.linspace(-50,150,150),'urad'), polarization = 'sigma')
print(scan)

The rocking curve is now calculated by initializing the TakagiTaupin instance with the crystal and scan parameter instances, and calling `.run()`.

In [ ]:
tt = TakagiTaupin(xtal, scan)

scan_vector, reflectivity, transmission = tt.run()

(Note that Jupyter does not update the solving process in the notebook but in the terminal/command prompt.)

`.run()` returns the solution but also stores it in `.solution` with all the relevant information and settings pertaining to it. The stored solution can be quickly visualized by calling `.plot()` and saved as an ASCII file with `.save(filename)` 

In [ ]:
tt.plot()

For reference, the same curve was calculated using _XCRYSTAL_ in _XOP_ 2.4 (http://dx.doi.org/10.1117/12.560903) with the EPDL97 anomalous scattering cross-sections, that are also used by _xraylib_. 

In [ ]:
ref = np.loadtxt('reference_spectra/xop_Si_111_bragg.dat')

plt.plot(scan_vector,reflectivity)
plt.plot(ref[:,0],ref[:,1])
plt.legend(['pyTTE','XCRYSTAL'])
plt.ylabel('Reflectivity')
plt.xlabel('Angle (urad)')
plt.show()

The result is practically identical with the one calculated with _pyTTE_. Nearly insignificant deviations are probably explained by slight differences in structure factors: 

_PyTTE_ output:
```
F0   :  (115.0489207774186+6.498559984990174j)
Fh   :  (46.915823878813626-40.417263893823474j)
Fb   :  (40.417263893823446+46.91582387881365j)
```

_Crystal Parameters_ of `XCRYSTAL`:
```
 Structure factor F(0,0,0) =  (  115.01669312000000     ,  6.5506672799999999     )
 Structure factor FH =  (  47.595073721453730     , -41.044406441453731     )
 Structure factor FH_BAR =  (  41.044406441453731     ,  47.595073721453730     )
```

### Second example: The Laue case and automatic scan limits

_PyTTE_ does not require explicit differentiation between the Bragg case (reflection geometry) and the Laue case (transmission geometry) since the diffraction geometry is determined automatically from the propagation direction of the diffracted beam. This can be controlled via the asymmetry angle. For a symmetric Laue case, asymmetry angle is set to 90 degrees:

In [ ]:
xtal = TTcrystal(crystal='Ge', hkl=[4,0,0], thickness=Quantity(100,'um'), asymmetry = Quantity(90,'deg'))

In the previous example we defined the scan points manually but it can be inconvenient to iterate good values for the scan limits, especially with bent crystals. To make the task easier, _pyTTE_ can try to determine the limits automatically by giving `TTscan` the number of scan points instead of a scan vector. 

In [ ]:
scan = TTscan(constant = Quantity(7,'keV'), scan = 150, polarization = 'sigma')
print(scan)

In [ ]:
tt = TakagiTaupin(xtal, scan)

scan_vector, diffracted, transmission = tt.run()
tt.plot()

Also in this case, the correspondence with _XCRYSTAL_ is practically perfect. 

In [ ]:
ref = np.loadtxt('reference_spectra/xop_Ge_400_laue_transmission.dat')
ref2 = np.loadtxt('reference_spectra/xop_Ge_400_laue_diffraction.dat')

plt.plot(scan_vector,transmission)
plt.plot(scan_vector,diffracted)
plt.plot(ref[:,0],ref[:,1])
plt.plot(ref2[:,0],ref2[:,1])
plt.legend(['pyTTE transmission', 'pyTTE diffraction', 'XCRYSTAL transmission', 'XCRYSTAL diffraction'])
plt.ylabel('Reflectivity')
plt.xlabel('Angle (urad)')
plt.show()

### Third example: bent crystals and energy scans

Deformation can be introduced by passing the meridional and sagittal bending radii (`Rx` and `Ry`, respectively) to TTcrystal. In the case of spherical bending, a single keyword `R` can be used.

In [ ]:
ttx = TTcrystal(crystal='Si', hkl=[6,6,0], thickness=Quantity(150,'um'), R = Quantity(100,'cm'), asymmetry = Quantity(5, 'deg'))
tts = TTscan(scan = 200, constant = Quantity(75, 'deg'), polarization = 'pi')

tt = TakagiTaupin(ttx,tts)

tt.run()
tt.plot()

By default _pyTTE_ attempts to calculate the toroidal/spherical/cylindrical bending using elastic constants from a built-in compliance matrix $S$ (Source: CRC Handbook of Chemistry and Physics, 82nd edition). 

For anisotropic crystals, _PyTTE_ has two different deformation models, selected with the TTcrystal keyword `fix_to_axis`. The default value is `'shape'` which assumes that the main axes of curvature are along the coordinate axes $x$ and $y$, and are given by `Rx` and `Ry`. This corresponds to the situation where the crystal is forced to a specific shape _e.g._ by the substrate. The other option is `torques` which assumes that the crystal is bent by two orthogonal torques acting about the aforemention coordinate axes. This applies, for example, when a free standing crystal slab is bent by its ends. In the general case these models are not equal due to non-diagonal elements of $S$. See TTcrystal.ipynb, help(TTcrystal) or the technical docs for more information.

Built-in $S$:s are available only for a handful of crystals commonly encountered in X-ray optics. In other cases user needs to define $S$ manually with the keyword `S`. Alternatively, an isotropic deformation model can be used by giving Poisson's ratio $\nu$ with `nu`. Also Young's modulus $E$ is required but its value is not important here as it is not used in by deformation model. `fix_to_axes` is neglected as the two anisotropic models reduce to the same isotropic model. 

In [ ]:
ttx = TTcrystal(crystal='Si', hkl=[6,6,0], thickness=Quantity(150,'um'), R = Quantity(100,'cm'), asymmetry = Quantity(5, 'deg'), nu = 0.22, E = Quantity(150, 'GPa'))
tts = TTscan(scan = 200, constant = Quantity(75, 'deg'), polarization = 'pi')

tt = TakagiTaupin(ttx,tts)

tt.run()
tt.plot()

Also custom deformation fields can be defined by providing the Jacobian of the displacement vector $\mathbf{u}$ as a function of $x$ and $z$:

$\begin{equation}
J(x,z) = \left[\begin{matrix}
\frac{\partial u_x}{\partial x}(x,z) & \frac{\partial u_x}{\partial z}(x,z) \\
\frac{\partial u_z}{\partial x}(x,z) & \frac{\partial u_z}{\partial z}(x,z)
\end{matrix}\right]
\end{equation}$

For reference, we do this for the isotropic bending below. Note that the lengths are exceptionally required to be in micrometers.

In [ ]:
def jacobian(x,z):
    nu = 0.22
    R_bend_in_um = 1e6
    thickness_in_um = 150

    ux_x = -(z+0.5*thickness_in_um)/R_bend_in_um 
    ux_z = -x/R_bend_in_um 

    uz_x = x/R_bend_in_um 
    uz_z = 2*nu/(1-nu)/R_bend_in_um*(z+0.5*thickness_in_um)

    return [[ux_x,ux_z],[uz_x,uz_z]]

ttx = TTcrystal(crystal='Si', hkl=[6,6,0], thickness=Quantity(150,'um'), asymmetry = Quantity(5, 'deg'))
tts = TTscan(scan = 200, constant = Quantity(75, 'deg'), polarization = 'pi')

#The custom Jacobian is given to TTcrystal after the initialization
ttx.set_deformation(jacobian = jacobian)

tt = TakagiTaupin(ttx,tts)

tt.run()
tt.plot()

### Fourth example: Initialization from file

Parameters can also be defined in two input files for `TTcrystal` and `TTscan` which can be passed directly to `TakagiTaupin`.

In [ ]:
'''
Contents of TTcrystal_init.inp:

crystal LiF
hkl 2 0 0
thickness 200 um
asymmetry 2.5 deg
Rx 1 m

Contents of TTscan_init.inp:

constant 8 keV
scan -100 25 150 arcsec
polarization sigma
solver zvode_bdf
integration_step 1 nm
'''

tt = TakagiTaupin('TTcrystal_init.inp','TTscan_init.inp')

print(tt)

tt.run()
tt.plot()
plt.show()